<a href="https://colab.research.google.com/github/zmohaghegh/CNN-Anomaly-Object-Detection/blob/main/Anomaly_Detection_using_CNN_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import cv2

# --- 1. LOAD AND PREPROCESS DATASET ---
# Using CIFAR-10: Training on class 1 (Automobiles) as 'Normal'
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# Filter only 'Automobile' class for training the model to recognize 'Normal'
normal_mask = np.where(y_train == 1)[0]
train_normal = x_train[normal_mask]

# --- 2. BUILD CNN AUTOENCODER MODEL ---
class AnomalyDetector(models.Model):
    def __init__(self):
        super(AnomalyDetector, self).__init__()
        # Encoder: Compressing image to latent space
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(32, 32, 3)),
            layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2),
            layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)
        ])

        # Decoder: Reconstructing image from latent space
        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Conv2DTranspose(32, kernel_size=3, strides=2, activation='relu', padding='same'),
            layers.Conv2D(3, kernel_size=(3, 3), activation='sigmoid', padding='same')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Compile model
autoencoder = AnomalyDetector()
autoencoder.compile(optimizer='adam', loss='mae')

# --- 3. TRAINING ---
print("🚀 Training Autoencoder on Normal images...")
autoencoder.fit(train_normal, train_normal,
                epochs=20,
                batch_size=64,
                validation_split=0.1)

# --- 4. CALCULATE THRESHOLD ---
reconstructions = autoencoder.predict(train_normal)
train_loss = np.mean(np.abs(reconstructions - train_normal), axis=(1, 2, 3))
threshold = np.mean(train_loss) + np.std(train_loss)
print(f"✅ Anomaly Threshold: {threshold:.4f}")

# --- 5. VISUALIZATION WITH HEATMAP ---
def plot_anomaly_heatmap(images, n=4):
    reconstructed = autoencoder.predict(images)

    # Calculate absolute difference for heatmap
    diff = np.abs(reconstructed - images)
    # Average across color channels to get intensity
    heatmap = np.mean(diff, axis=-1)

    # Final Score (MAE)
    scores = np.mean(diff, axis=(1, 2, 3))

    plt.figure(figsize=(16, 10))
    for i in range(n):
        is_anomaly = scores[i] > threshold
        color = 'red' if is_anomaly else 'green'
        status = "ANOMALY" if is_anomaly else "NORMAL"

        # 1. Original Image
        plt.subplot(3, n, i + 1)
        plt.imshow(images[i])
        plt.title(f"Original\nResult: {status}", color=color)
        plt.axis('off')

        # 2. Reconstructed Image
        plt.subplot(3, n, i + 1 + n)
        plt.imshow(reconstructed[i])
        plt.title(f"Reconstructed\nScore: {scores[i]:.4f}")
        plt.axis('off')

        # 3. Anomaly Heatmap
        plt.subplot(3, n, i + 1 + 2*n)
        plt.imshow(heatmap[i], cmap='jet') # 'jet' shows hot spots in red
        plt.title("Anomaly Heatmap")
        plt.colorbar(fraction=0.046, pad=0.04)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# --- 6. FINAL TEST ---
# Testing with mixed samples (1 Normal Car vs 3 Anomalies like Birds/Dogs)
car_idx = np.where(y_test == 1)[0][0]
anomaly_indices = np.where(y_test != 1)[0][:3]
test_indices = [car_idx] + list(anomaly_indices)
test_samples = x_test[test_indices]

plot_anomaly_heatmap(test_samples)